(PERFBLAS)=

# 5.2 Herramientas de lenguajes de programación y del sistema operativo para perfilamiento e implementaciones de BLAS

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

```{admonition} Al final de esta nota el y la lectora:
:class: tip

* 

```

En esta nota revisamos algunas herramientas de los lenguajes de programación y de los sistemas GNU/Linux para perfilamiento de código. También se revisan implementaciones de la especificación de *Basic Linear Algebra Subprograms* (BLAS) en sus diferentes niveles con ejemplos de paquetes en los lenguajes.

```{margin}

Utilizamos la máquina `m4.16xlarge` pues en AWS no todas las instancias se pueden obtener algunas métricas con `perf`. Ver [stackoverflow1](https://stackoverflow.com/questions/45683901/event-cache-misses-not-supported-by-perf-in-aws), [stackoverflow2](https://stackoverflow.com/questions/19763070/ubuntu-12-10-perf-stat-not-supported-cycles), [the-pmcs-of-ec2](http://www.brendangregg.com/blog/2017-05-04/the-pmcs-of-ec2.html)

```

Se presentan códigos y sus ejecuciones en una máquina `m4.16xlarge` de la nube de [AWS](https://aws.amazon.com/). Se utilizó la AMI `opt2-aws-educate-openblas-04-04-2021` de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

La máquina `m4.16xlarge` tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2709.718
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.02
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        8

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.11.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 256GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits
     *-bank:2
          description: DIMM RAM
          physical id: 2
          slot: DIMM 2
          size: 16GiB
          width: 64 bits
     *-bank:3
          description: DIMM RAM
          physical id: 3
          slot: DIMM 3
          size: 16GiB
          width: 64 bits
     *-bank:4
          description: DIMM RAM
          physical id: 4
          slot: DIMM

```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con *import*'s. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

## Perfilamiento: medición de tiempo en *Python*

### Módulo: [time](https://docs.python.org/3/library/time.html#time.time)

Imports y funciones de apoyo:

In [1]:
import math
import time

import numpy as np
from pytest import approx
from scipy.integrate import quad

In [4]:
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res


Objetivo:

In [5]:
f=lambda x: math.exp(-x**2) #using math library
a=0
b=1
obj, err = quad(f, a, b)
print(obj)

0.7468241328124271


Medición de tiempo:

In [6]:
n=10**6
start_time = time.time()
res=Rcf(f,a,b,n)
end_time = time.time()
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.44214773178100586 segundos


Prueba que se resuelve correctamente el problema:

In [7]:
print(res == approx(obj))

True


### Comando de *magic*: [%time](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time)

Nos regresa las mediciones siguientes:

* `CPU times` que contiene:

    * `user`: mide la cantidad de tiempo de los *statements* que la CPU gastó para funciones que no están relacionadas con el *kernel* del sistema.
    
    * `sys`: mide la cantidad de tiempo de los *statements* que la CPU gastó en funciones a nivel de kernel del sistema.
    
    * `total`: suma entre el `user` y `sys` para todos **todos los *cores***. 
    
* `Wall time`: mide el *wall clock* o *elapsed time* que se refiere al tiempo desde que inicia la ejecución de los *statements* hasta su finalización.

* `Out`: resultado.



In [8]:
%time Rcf(f,a, b,n)

CPU times: user 365 ms, sys: 22.9 ms, total: 388 ms
Wall time: 386 ms


0.7468241328124773

```{admonition} Comentarios

* Para mediciones de tiempos que involucran cómputo en paralelo es posible que `total` exceda a `Wall time` por una diferencia notable pues es la suma de tiempos para todos los *cores*.

* Recuérdese que algunos ejemplos de funciones relacionadas con el *kernel* del sistema es el alojamiento, lectura y escritura de variables en memoria, las relacionadas con el I/O de disco o *network*.

* La diferencia entre `total` y `Wall time` da una idea de la cantidad de tiempo que se ocupó el sistema en la ejecución de *statements* que no involucran `user` ni a `sys` (por ejemplo un *statement* del tipo `sleep`) o en tareas no relacionadas con los *statements* medidos (por ejemplo si en el momento de la medición se estaban corriendo otros procesos).

```

Al medir tiempos de ejecución, **siempre** hay variación en la medición por lo que se recomienda realizar las mediciones un número repetido de veces. Para este caso el comando de *magic* `%timeit` nos ayuda. 

### Comando de *magic*: [%timeit](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)

A continuación se mide el tiempo de ejecución para la función `Rcf`. Se promedian los tiempos de $n=5$ ejecuciones y se calcula su desviación estándar. Se reptite lo anterior $r=10$ veces y se reporta el mejor resultado. 

In [9]:
%timeit -n 5 -r 10 Rcf(f,a, b,n)

369 ms ± 1.99 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


```{admonition} Observación
:class: tip

$ms$ es milisecond, $\mu s$ es microsecond y $ns$ es nanosecond.

```

```{admonition} Comentarios

* `%timeit` se recomienda usar para secciones de código pequeñas. Si se observa una variación en las mediciones de tiempo entre distintas repeticiones entonces hay que realizar más repeticiones hasta tener un resultado estable.

* `%timeit` desabilita temporalmente el *garbage collector* de *Python* (esto es, no habrá desalojamiento en memoria de objetos de Python que no se utilicen). Si el *garbage collector* es invocado en tus *statements*, esto puede ser una razón de posibles diferencias que se obtengan en las mediciones de tiempo.


```

## Perfilamiento: medición de uso de CPU en *Python*

### [line_profiler](https://pypi.org/project/line-profiler/)

### [CProfile](https://docs.python.org/2/library/profile.html) 

*CProfile* es `built-in` en la *standard-library* de Python.

## Perfilamiento: medición de uso de memoria en *Python*

### [memory_profiler](https://pypi.org/project/memory-profiler/)

### [heapy](https://pypi.org/project/guppy/)

## *Basic Linear Algebra Subprograms* (BLAS)

De acuerdo a la especificación BLAS, hay diferentes niveles dependiendo del número de datos y cantidad de operaciones a realizar que involucran tales datos.

### Operación del producto interno estándar o producto punto (nivel 1 de BLAS)

Consideramos $x,y \in \mathbb{R}^n$. El producto punto entre $x$ y $y$ es $c = x^Ty = \displaystyle \sum_{i=1}^n x_iy_i$. 

### Ejemplo y algoritmo

In [1]:
c=0
n=5
x=[-1]*n
y=[1.5]*n

for i in range(n):
    c += x[i]*y[i]

In [2]:
c

-7.5

````{admonition} Comentarios

* El producto punto de dos $n$-vectores involucran $n$ multiplicaciones y $n$ sumas para un total de $2n$ operaciones o [floating point operations per second](https://en.wikipedia.org/wiki/FLOPS) (flops). Usamos la notación $\mathcal{O}(\cdot)$ para escribir que el producto punto es $\mathcal{O}(n)$ y se lee "de orden $n$ o proporcional a $n$" para indicar que la **cantidad de trabajo** tiene un comportamiento **lineal** con la dimensión $n$. También tal cantidad de trabajo opera sobre una **cantidad lineal de datos**.


* Los *flops* que realiza un algoritmo es una forma de cuantificar el volumen de trabajo asociado con un cálculo. Un *flop* es una operación de punto flotante: suma, multiplicación o división. Por ejemplo, en la línea:

```python
C[i][j] = C[i][j] + A[i][k]*B[k][j]
```

se realizan $2$ *flops*. Los flops sólo representan una componente para categorizar a los algoritmos de acuerdo al trabajo que realizan, otras componentes son la transferencia o movimientos de datos, *data movement/motion*, ejecución secuencial o en paralelo y el *data locality* y *data reuse* que realizan.


* En LAPACK encontramos [sdot](http://www.netlib.org/lapack/explore-html/d0/d16/sdot_8f.html), [ddot](http://www.netlib.org/lapack/explore-html/d5/df6/ddot_8f.html), [cdotu](http://www.netlib.org/lapack/explore-html/d7/d7b/cdotu_8f.html) y [zdotu](http://www.netlib.org/lapack/explore-html/db/d2d/zdotu_8f.html) para descripción de las funciones/subrutinas escritas en [Fortran](https://en.wikipedia.org/wiki/Fortran) del producto punto en los casos de precisión simple, doble o números complejos respectivamente.

````

### Operación **saxpy** (nivel 1 de BLAS)

Consideramos $\alpha \in \mathbb{R}, x,y \in \mathbb{R}^n$. El nombre lo recibe por *scalar alpha x plus y*. En LAPACK [saxpy](http://www.netlib.org/lapack/explore-html/d8/daf/saxpy_8f.html) se escribe en forma *update*:

$$y=\alpha x + y \therefore y_i = \alpha x_i + y_i \forall i=1,...,n$$

```{admonition} Comentarios

* El símbolo "=" no se utiliza como igualdad de expresiones sino para denotar asignación (como en computación al escribir un algoritmo).

* También encontramos en LAPACK [caxpy](http://www.netlib.org/lapack/explore-html/de/da2/caxpy_8f.html) o [daxpy](http://www.netlib.org/lapack/explore-html/d9/dcd/daxpy_8f.html) para el caso complejo y para números en doble precisión respectivamente.

* Ésta operación realiza un trabajo de $\mathcal{O}(n)$ sobre una cantidad de datos $\mathcal{O}(n)$.

```

### Ejemplo y algoritmo

In [3]:
alpha=2
n=5
x=[-2]*n
y=[0]*n

for i in range(n):
    y[i] += alpha*x[i]

In [4]:
print(y)

[-4, -4, -4, -4, -4]


o en una forma *update*:

In [5]:
alpha=2
n=5
x=[-2]*n
y=[3,4,-1,0,1]

for i in range(n):
    y[i] += alpha*x[i]

In [6]:
print(y)

[-1, 0, -5, -4, -3]


```{admonition} Comentario

La operación de producto punto y *saxpy* son algoritmos catalogados como de **nivel BLAS 1**, ver [BLAS: Basic Linear Algebra Subprograms](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms). Éstos algoritmos se caracterizan por involucrar una cantidad de trabajo lineal sobre una cantidad lineal de datos. Ver [level 1](http://www.netlib.org/blas/#_level_1) para más ejemplos de este tipo de algoritmos.

```

### Operación de multiplicación matriz-vector (nivel 2 de BLAS)

Consideramos $A \in \mathbb{R}^{m \times n}, x \in \mathbb{R}^n, y \in \mathbb{R}^m$. La operación $y = y + Ax$ es una operación *generalizada* saxpy, por ello se denomina **gaxpy** pero en LAPACK podemos encontrarla con nombres como [sgemv](http://www.netlib.org/lapack/explore-html/db/d58/sgemv_8f.html), [dgemv](http://www.netlib.org/lapack/explore-html/dc/da8/dgemv_8f.html), [cgemv](http://www.netlib.org/lapack/explore-html/d4/d8a/cgemv_8f.html) o [zgemv](http://www.netlib.org/lapack/explore-html/db/d40/zgemv_8f.html) para los casos de precisión simple, doble o números complejos respectivamente. Hay diferentes formas de visualizar y escribir el algoritmo de multiplicación matriz-vector. Por ejemplo para una matriz $A$ con entradas:


In [7]:
m=2
n=5
A=[[1.2]*n if i%2==0 else [1]*n for i in range(m)]

In [8]:
print(A)

[[1.2, 1.2, 1.2, 1.2, 1.2], [1, 1, 1, 1, 1]]


se tiene:

### Ejemplo y algoritmo gaxpy *row oriented*

In [9]:
x=[2]*n
y=[0]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [10]:
print(y)

[12.0, 10]


Si $y$ tiene valores distintos de $0$, se realiza un *update*:

In [11]:
x=[2]*n
y=[-1]*m
for i in range(m):
    for j in range(n):
        y[i]+=A[i][j]*x[j]


In [12]:
print(y)

[11.0, 9]


````{admonition} Comentarios

* En la versión *row oriented* del algoritmo *gaxpy*, el **inner loop** realiza **productos punto** entre el $i$-ésimo renglón de $A$ y el vector $x$. Se realizan $m$ productos punto $A[i,:]^Tx$

```python
for i in range(m):
    y[i]+=A[i,:]*x #producto punto
    
```

donde: $A[i,:]$ es el $i$-ésimo renglón de $A$. Así podemos reescribir de forma más compacta este algoritmo.

Sin embargo como hemos visto en Python con su implementación más común CPython, no es posible realizar tal indexado pues resulta en el error:

```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-14-8ae2a3e9d101> in <module>
      1 for i in range(m):
----> 2     y[i]+=A[i,:]*x #producto punto
      3 

TypeError: list indices must be integers or slices, not tuple
```

a menos que incorporemos alguna paquetería que permita la **vectorización** y el uso de índices para extracción de columnas (o renglones) de $A$ como *NumPy*.

````

```{admonition} Observación
:class: tip

Obsérvese que el acceso a la matriz $A$ del algoritmo *gaxpy row oriented* es **por renglón**, de ahí el nombre.

```

También puede escribirse al algoritmo *gaxpy* en una forma orientada por columnas.

### Ejemplo y algoritmo gaxpy *column oriented*

Este algoritmo ayuda a visualizar al producto matriz-vector como una combinación lineal de las columnas de $A$:

$$Ax = \displaystyle \sum_{j=1}^n A_jx_j$$

con $A_j$ la $j$-ésima columna de $A$.


In [17]:
x=[2]*n
y=[0]*m
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

In [18]:
print(y)

[12.0, 10]


````{admonition} Comentarios 

* El algoritmo de multiplicación matriz-vector (versión *row* o *column* oriented) involucra $\mathcal{O}(mn)$ operaciones o una cantidad **cuadrática** de trabajo, que podemos entender como "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $4$". Tal número de operaciones trabajan sobre una matriz o sobre una cantidad **cuadrática** de datos. A los algoritmos que realizan una cantidad cuadrática de trabajo sobre una cantidad cuadrática de datos se les cataloga de **nivel BLAS 2**. Ver [level 2](http://www.netlib.org/blas/#_level_2) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

* En el algoritmo *gaxpy column oriented* el acceso a la matriz $A$ es por columna.

* La versión *column oriented* se puede analizar desde el punto de vista puramente algorítmico como un intercambio entre las líneas con los índices $i$ y $j$ de cada *loop* y un acceso a los datos de la matriz por columna. O bien, se puede analizar desde el álgebra lineal indicando que el vector $y$ está en el espacio generado por las columnas de $A$ y cuyas coordenadas son dadas por las entradas del vector $x$:

<img src="https://dl.dropboxusercontent.com/s/6a2b7rjs4a71sni/combinacion_lineal_columnas_A.png?dl=0" heigth="700" width="700">

* Obsérvese que el **inner loop** de la versión *column oriented* en *gaxpy* es un **saxpy** en la que el escalar está dado por una entrada de $x$. Esto lo podemos escribir de forma explícita definiendo $A[:,j]$ a la $j$-ésima columna de $A$ por lo que $A = [A[:,1] | A[:,2] | \dots | A[:,n]]$, entonces:

```python
x=[2]*n
y=[0]*m
for j in range(n):
    y+=A[:,j]*x[j]
```

Sin embargo como hemos visto en Python con su implementación más común CPython, no es posible realizar tal indexado pues resulta en el error:

```python
TypeError                                 Traceback (most recent call last)
<ipython-input-17-93f449af9194> in <module>
      2 y=[0]*m
      3 for j in range(n):
----> 4     y+=A[:,j]*x[j]

TypeError: list indices must be integers or slices, not tuple

```

a menos que incorporemos alguna paquetería que permita la **vectorización** y el uso de índices para extracción de columnas (o renglones) de $A$ como *NumPy*.

````

In [17]:
x = 2*np.ones(n)
y = np.zeros(m)

In [20]:
print(x)

[2. 2. 2. 2. 2.]


In [21]:
print(y)

[0. 0.]


In [22]:
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [23]:
print(A)

[[1.2 1.2 1.2 1.2 1.2]
 [1.  1.  1.  1.  1. ]]


In [24]:
for j in range(n):
    y+=A[:,j]*x[j]

In [25]:
print(y)

[12. 10.]


El algoritmo *gaxpy row oriented* puede escribirse de forma más compacta haciendo uso de la definición de producto punto estándar: $x^Ty$ para dos vectores columna $x$ y $y$. En el caso de una matriz $A$ se tiene:

```
for i=1:m
    y[i]+=A[i,:]^T*x
```

donde: $A[i,:]$ es el $i$-ésimo renglón de $A$. En *Python*:

In [26]:
x = 2*np.ones(n)
y = np.zeros(m)
A=np.array([[1.2,1.2,1.2,1.2,1.2],[1,1,1,1,1]])

In [27]:
for i in range(m):
    y[i]+=A[i,:].dot(x)

In [28]:
print(y)

[12. 10.]


### OpenBLAS y *NumPy*

En esta sección se sugiere para reproducibilidad de resultados utilizar la *AMI* y la instancia de AWS ambas descritas al inicio y la *AMI* `opt2-aws-educate-no-openblas-04-04-2021` de la región `us-east-1` (Virginia) en el mismo tipo de instancia. Tal AMI se construyó a partir de una *AMI* `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_no_OpenBLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_no_OpenBLAS.sh)


````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-no-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=no-OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

### Resultados utilizando `opt2-aws-educate-openblas-04-04-2021`

Verificamos que *NumPy* está usando la implementación de la API de BLAS con OpenBLAS que se obtiene con `sudo apt-get install -y libopenblas-dev`, ver [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

In [10]:
print(np.show_config())

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/lib/x86_64-linux-gnu']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
None


Hacemos una prueba de medición de tiempos:

In [11]:
np.random.seed(2020)
m=10**4
r=10**4

A=np.random.rand(m,r)

In [12]:
np.random.seed(2021)
r=10**4
n=10**4

B=np.random.rand(r,n)

```{admonition} Observación
:class: tip

Mientras se ejecuta la siguiente celda se sugiere en la terminal ejecutar en la línea de comando `htop`.

```

In [13]:
%timeit -n 1 -r 7 A@B

2.96 s ± 149 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


### Resultados utilizando `opt2-aws-educate-no-openblas-04-04-2021`

Verificamos que *NumPy* **no** está usando la implementación de la API de BLAS con OpenBLAS que se obtiene con `sudo apt-get install -y libopenblas-dev`, ver [script_no_OpenBLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_no_OpenBLAS.sh)

In [2]:
print(np.show_config())

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
None


Hacemos una prueba de medición de tiempos:

In [3]:
np.random.seed(2020)
m=10**4
r=10**4

A=np.random.rand(m,r)

In [4]:
np.random.seed(2021)
r=10**4
n=10**4

B=np.random.rand(r,n)

```{admonition} Observación
:class: tip

Mientras se ejecuta la siguiente celda se sugiere en la terminal ejecutar en la línea de comando `htop`.

```

In [5]:
%timeit -n 1 -r 7 A@B

3.22 s ± 167 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


```{admonition} Comentarios

* La implementación de la API de BLAS con OpenBLAS que se instala mediante `sudo apt-get install -y libopenblas-dev` y *NumPy* instalado vía `pip install numpy --no-binary numpy` resulta para el algoritmo de multiplicación de matrices ser un poco más rápido que con la implementación de la API de BLAS integrada en la instalación de *NumPy* vía `pip install numpy`.

* Más *cores* reducen el tiempo de cómputo, para ver esto probar lo anterior con máquinas con menos *cores*.

```

### OpenBLAS y librerías de cómputo matricial de *R*

En esta sección se sugiere para reproducibilidad de resultados utilizar la *AMI* y la instancia de AWS ambas descritas al inicio y la *AMI* `opt2-aws-educate-no-openblas-04-04-2021` de la región `us-east-1` (Virginia) en el mismo tipo de instancia. Tal AMI se construyó a partir de una *AMI* `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_no_OpenBLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_no_OpenBLAS.sh)


````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-no-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=no-OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

### Resultados utilizando `opt2-aws-educate-openblas-04-04-2021`

Verificamos que las librerías de cómputo matricial de *R* están usando la implementación de la API de BLAS con OpenBLAS que se obtiene con `sudo apt-get install -y libopenblas-dev`, ver [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

In [1]:
sessionInfo()

R version 4.0.5 (2021-03-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] fansi_0.4.2         digest_0.6.27       utf8_1.2.1         
 [4] crayon_1.4.1        IRdisplay_1.0       repr_1.1.3         
 [7] lifecycle_1.0.0     jsonlite_1.7.2      evaluate_0.14      
[10] pillar_1.5.1        rlang_0.4.10        uuid_0.1-4         
[13] ellipsis_0.3.1      IRkernel_

In [2]:
set.seed(2020)
m<-10**4
r<-10**4
A <- matrix(runif(m*r), m, r)

In [3]:
set.seed(2021)
r<-10**4
n<-10**4
B <- matrix(runif(r*n), r, n)

In [5]:
library(microbenchmark)

```{admonition} Comentario

El paquete de [microbenchmark](https://www.rdocumentation.org/packages/microbenchmark/versions/1.4-7/topics/microbenchmark) nos ayuda a la medición de estadísticas para pequeñas piezas de código.

```

```{admonition} Observación
:class: tip

Mientras se ejecuta la siguiente celda se sugiere en la terminal ejecutar en la línea de comando `htop`.

```

In [10]:
mbk<-microbenchmark(
    A%*%B,
    times=7
    )

In [11]:
print(mbk)

Unit: seconds
    expr      min       lq     mean   median       uq      max neval
 A %*% B 2.600818 2.723897 2.952135 2.833146 3.038394 3.706399     7


### Resultados utilizando `opt2-aws-educate-no-openblas-04-04-2021`

Verificamos que las librerías de cómputo matricial de *R* **no** están usando la implementación de la API de BLAS con OpenBLAS que se obtiene con `sudo apt-get install -y libopenblas-dev`, ver [script_no_OpenBLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_no_OpenBLAS.sh)

In [1]:
sessionInfo()

R version 4.0.5 (2021-03-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.9.0
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.9.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] fansi_0.4.2         digest_0.6.27       utf8_1.2.1         
 [4] crayon_1.4.1        IRdisplay_1.0       repr_1.1.3         
 [7] lifecycle_1.0.0     jsonlite_1.7.2      evaluate_0.14      
[10] pillar_1.5.1        rlang_0.4.10        uuid_0.1-4         
[13] ellipsis_0.3.1      IRkernel_1.1.1.9000 too

In [2]:
set.seed(2020)
m<-10**4
r<-10**4
A <- matrix(runif(m*r), m, r)

In [3]:
set.seed(2021)
r<-10**4
n<-10**4
B <- matrix(runif(r*n), r, n)

In [4]:
library(microbenchmark)

```{admonition} Observación
:class: tip

Mientras se ejecuta la siguiente celda se sugiere en la terminal ejecutar en la línea de comando `htop`.

```

In [ ]:
mbk<-microbenchmark(
    A%*%B,
    times=1 #just one time as it takes too much time
    )

In [ ]:
print(mbk) #after ten minutes didnt finish

```{admonition} Comentarios

* La implementación de la API de BLAS con OpenBLAS que se instala mediante `sudo apt-get install -y libopenblas-dev` y las librerías de cómputo matricial de *R* que utilizan tal instalación resulta para el algoritmo de multiplicación de matrices ser mucho más rápido que con la implementación de la API de BLAS integrada en la instalación de *R*.

* Más *cores* reducen el tiempo de cómputo, para ver esto probar lo anterior con máquinas con menos *cores*.

```

## Perfilamiento: medición de tiempo en el sistema operativo Ubuntu 20.04

### [/usr/bin/time](http://manpages.ubuntu.com/manpages/focal/man1/time.1.html)

Ver también: [Wikipedia: Time_Unix](https://en.wikipedia.org/wiki/Time_(Unix))


In [18]:
%%file Rcf.py
import math

from pytest import approx
from scipy.integrate import quad
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res
if __name__=="__main__": #añadimos este bloque para ejecución de la función Rcf
    a=0
    b=1
    f=lambda x: math.exp(-x**2)
    obj, err = quad(f, a, b)
    print("objetivo: {:0.6e}".format(obj))
    n=10**6
    res=Rcf(f,a,b,n)
    print("aproximación: {:0.6e}".format(res))
    print(res == approx(obj))

Writing Rcf.py


In [19]:
%%bash
/usr/bin/time -p python3 Rcf.py

objetivo: 7.468241e-01
aproximación: 7.468241e-01
True


real 0.84
user 2.57
sys 2.20


In [20]:
%%bash
/usr/bin/time -v python3 Rcf.py

objetivo: 7.468241e-01
aproximación: 7.468241e-01
True


	Command being timed: "python3 Rcf.py"
	User time (seconds): 2.55
	System time (seconds): 2.34
	Percent of CPU this job got: 612%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.79
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 97688
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 20846
	Voluntary context switches: 87
	Involuntary context switches: 877104
	Swaps: 0
	File system inputs: 0
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


```{admonition} Comentario

Una breve explicación del *output* se puede encontrar [/usr/bin/time](http://manpages.ubuntu.com/manpages/focal/man1/time.1.html). Por ejemplo, para el caso de `Major (requiring I/O)` nos interesa que sea $0$ pues indica que el sistema operativo tiene que cargar páginas de datos del disco pues tales datos ya no residen en RAM (por alguna razón).

```

```{admonition} Observación
:class: tip

La medición `real` que regresa `/usr/bin/time` es el equivalente al `Wall time` de `%time`.


```

## Perfilamiento: medición de uso de CPU en el sistema operativo GNU/Linux

### [perf](https://github.com/torvalds/linux/tree/master/tools/perf)

En GNU/Linux existe la herramienta `perf` que nos ayuda a calcular métricas de desempeño de la CPU. Los siguientes ejemplos se ejecutaron con la AMI `opt2-aws-educate-openblas-04-04-2021` y la instancia de AWS, ambas descritas al inicio.

### perf OpenBLAS

In [14]:
%%file mult_matrix_matrix_numpy_openblas.py
import numpy as np
m=10**4
r=10**4
n=10**4

np.random.seed(2020)
m=10**4
r=10**4
A=np.random.rand(m,r)

np.random.seed(2021)
r=10**4
n=10**4
B=np.random.rand(r,n)

A@B

Writing mult_matrix_matrix_numpy_openblas.py


```{margin}

Esta línea de `echo` es importante para permitir que *user* `ubuntu` ejecute `perf` sin `sudo`.

```

In [15]:
%%bash
echo "-1" |sudo tee -a /proc/sys/kernel/perf_event_paranoid

-1


In [16]:
%%bash
perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 7 python3 mult_matrix_matrix_numpy_openblas.py


 Performance counter stats for 'system wide' (7 runs):

S0-D0-C0           2        18418711959      cycles                                                        (75.00%)
S0-D0-C0           2        18369720566      instructions              #    1.00  insn per cycle           (75.06%)
S0-D0-C0           2          537701642      cache-references                                              (75.06%)
S0-D0-C0           2           24051039      cache-misses              #    4.473 % of all cache refs      (75.02%)
S0-D0-C1           2        18196672426      cycles                                                        (75.03%)
S0-D0-C1           2        18408825312      instructions              #    1.01  insn per cycle           (75.06%)
S0-D0-C1           2          525743359      cache-references                                              (75.07%)
S0-D0-C1           2           23125192      cache-misses              #    4.399 % of all cache refs      (75.01%)
S0-D0-C2       

### Otras herramientas para medición de uso de CPU en el sistema operativo GNU/Linux

* [pmc-cloud-tools](https://github.com/brendangregg/pmc-cloud-tools)

```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Referencias:**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

2. E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

